In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

**Import Data**

In [ ]:
building_metadata = pd.read_csv('/kaggle/input/ashrae-energy-prediction/building_metadata.csv')
train = pd.read_csv('/kaggle/input/ashrae-energy-prediction/train.csv')
test = pd.read_csv('/kaggle/input/ashrae-energy-prediction/test.csv')
weather_train = pd.read_csv('/kaggle/input/ashrae-energy-prediction/weather_train.csv')
weather_test = pd.read_csv('/kaggle/input/ashrae-energy-prediction/weather_test.csv')

In [ ]:
print('Size of the building_metadata dataset is', building_metadata.shape)
print('Size of the weather_train dataset is', weather_train.shape)
print('Size of the train dataset is', train.shape)
print('Size of the weather_test dataset is', weather_test.shape)
print('Size of the test dataset is', test.shape)

**Explore Train Data**

In [ ]:
train.head()

In [ ]:
print('The start date is', train.timestamp[0])
print('The last date is', train.timestamp[len(train) -1])

In [ ]:
train.describe()

In [ ]:
train.info()

Observations from train dataset:

1. The data is taken from Jan 1 2016 to Dec 31 2016
2. There are 2,021,6100 data points in the  dataset
3. The meter_reading is the target value and ranges from 0 to 21 million
4. The 'meter' column is categorical, refers to the meter type, and is assigned an integer value from 0 to 3
5. From ASHRAE EDA, we have the following id assignments for meter type: {0: electricity, 1: chilledwater, 2: steam, hotwater: 3} 
6. There are no missing data values 

 **Explore Test Data**

In [ ]:
print('The start date is', test.timestamp[0])
print('The last date is', test.timestamp[len(test) -1])

In [ ]:
test.info()

Observations from the test dataset

1. There are 41,697,600 test data points.
2. The test data is taken from Jan 1 2017 to May 9 2018 
3. There are no missing values in the dataset

**Explore Building Metadata**

In [ ]:
building_metadata.head()

In [ ]:
building_metadata.info()

In [ ]:
# Also, each of the 1449 buildings has a site_id. We can determine the actual number of sites:
print(building_metadata.site_id.unique())
print(building_metadata.site_id.value_counts().sort_index())

Observations from building_metadata

1. There are a total of 1449 buildings/data points
2. There are 16 different sites for the 1449 buildings
3. Columns 'year_built' and 'floor_count' are missing values about 50% of values
4. The 'primary use' column is categorical  

**Explore weather_train**

In [ ]:
weather_train.head()

In [ ]:
print('The start date is', weather_train.timestamp[0])
print('The last date is', weather_train.timestamp[len(weather_train) -1])

In [ ]:
weather_train.info()

Observations from weather_train

1. Data is taken hourly from Jan 1 2016 to Dec 31 2016
2. There are a total of 139,773 data points. Since there are 16 sites and the data is taken hourly in the year 2016, there should be (16)(24)(366) = 140,544 data points. Yet, there are only 139,733 data points, which means we are missing hourly data. 
3. There is also missing data from every weather feature
4. The following units are given from the ASHRAE EDA:
    * air_temperature - Degrees Celsius
    * cloud_coverage - Portion of the sky covered in clouds, in oktas
    * dew_temperature - Degrees Celsius
    * precip_depth_1_hr - Millimeters
    * sea_level_pressure - Millibar/hectopascals
    * wind_direction - Compass direction (0-360)
    * wind_speed - Meters per second


**Explore Weather_test**

In [ ]:
print('The start date is', weather_test.timestamp[0])
print('The last date is', weather_test.timestamp[len(weather_test) -1])

In [ ]:
weather_test.info()

Observations from weather_test

1. Data is taken from Jan 1 2017 to Dec 31 2018
2. There are 277243 data points and there should be 280,320 data points, so there is missing hourly data
3. There is also missing data for every weather feature

**Data Cleaning**

In [ ]:
# Elminiate missing values from weather datasets by replacing NaN values with mean values taken for a month. 
# Source code taken from, https://www.kaggle.com/aitude/ashrae-missing-weather-data-handling#How-to-Use

def missing_statistics(df):    
    statitics = pd.DataFrame(df.isnull().sum()).reset_index()
    statitics.columns=['COLUMN NAME',"MISSING VALUES"]
    statitics['TOTAL ROWS'] = df.shape[0]
    statitics['% MISSING'] = round((statitics['MISSING VALUES']/statitics['TOTAL ROWS'])*100,2)
    return statitics
    
def fill_weather_dataset(weather_df):
    
    # Find Missing Dates
    time_format = "%Y-%m-%d %H:%M:%S"
    start_date = datetime.datetime.strptime(weather_df['timestamp'].min(),time_format)
    end_date = datetime.datetime.strptime(weather_df['timestamp'].max(),time_format)
    total_hours = int(((end_date - start_date).total_seconds() + 3600) / 3600)
    hours_list = [(end_date - datetime.timedelta(hours=x)).strftime(time_format) for x in range(total_hours)]

    missing_hours = []
    for site_id in range(16):
        site_hours = np.array(weather_df[weather_df['site_id'] == site_id]['timestamp'])
        new_rows = pd.DataFrame(np.setdiff1d(hours_list,site_hours),columns=['timestamp'])
        new_rows['site_id'] = site_id
        weather_df = pd.concat([weather_df,new_rows])

        weather_df = weather_df.reset_index(drop=True)           

    # Add new Features
    weather_df["datetime"] = pd.to_datetime(weather_df["timestamp"])
    weather_df["day"] = weather_df["datetime"].dt.day
    weather_df["week"] = weather_df["datetime"].dt.week
    weather_df["month"] = weather_df["datetime"].dt.month
    
    # Reset Index for Fast Update
    weather_df = weather_df.set_index(['site_id','day','month'])

    air_temperature_filler = pd.DataFrame(weather_df.groupby(['site_id','day','month'])['air_temperature'].mean(),columns=["air_temperature"])
    weather_df.update(air_temperature_filler,overwrite=False)

    # Step 1
    cloud_coverage_filler = weather_df.groupby(['site_id','day','month'])['cloud_coverage'].mean()
    # Step 2
    cloud_coverage_filler = pd.DataFrame(cloud_coverage_filler.fillna(method='ffill'),columns=["cloud_coverage"])

    weather_df.update(cloud_coverage_filler,overwrite=False)

    due_temperature_filler = pd.DataFrame(weather_df.groupby(['site_id','day','month'])['dew_temperature'].mean(),columns=["dew_temperature"])
    weather_df.update(due_temperature_filler,overwrite=False)

    # Step 1
    sea_level_filler = weather_df.groupby(['site_id','day','month'])['sea_level_pressure'].mean()
    # Step 2
    sea_level_filler = pd.DataFrame(sea_level_filler.fillna(method='ffill'),columns=['sea_level_pressure'])

    weather_df.update(sea_level_filler,overwrite=False)

    wind_direction_filler =  pd.DataFrame(weather_df.groupby(['site_id','day','month'])['wind_direction'].mean(),columns=['wind_direction'])
    weather_df.update(wind_direction_filler,overwrite=False)

    wind_speed_filler =  pd.DataFrame(weather_df.groupby(['site_id','day','month'])['wind_speed'].mean(),columns=['wind_speed'])
    weather_df.update(wind_speed_filler,overwrite=False)

    # Step 1
    precip_depth_filler = weather_df.groupby(['site_id','day','month'])['precip_depth_1_hr'].mean()
    # Step 2
    precip_depth_filler = pd.DataFrame(precip_depth_filler.fillna(method='ffill'),columns=['precip_depth_1_hr'])

    weather_df.update(precip_depth_filler,overwrite=False)

    weather_df = weather_df.reset_index()
    weather_df = weather_df.drop(['datetime','day','week','month'],axis=1)
        
    return weather_df

In [ ]:
missing_statistics(weather_train)

In [ ]:
missing_statistics(weather_test)

In [ ]:
weather_train = fill_weather_dataset(weather_train)
weather_test = fill_weather_dataset(weather_test)

In [ ]:
# Missing Data from building_metadata
missing_statistics(building_metadata)

In [ ]:
# floor count is missing too many values, probably best to delete it all together

del building_metadata['floor_count']
del building_metadata['year_built']

**Reduce Memory**

In [ ]:
# Since the dataset sizes are quite large, it is important to reduce memory usage before proceeding
# Source code taken from https://www.kaggle.com/cereniyim/save-the-energy-for-the-future-1-detailed-eda @ 
  
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype


def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")
    
    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
building_metadata = reduce_mem_usage(building_metadata,use_float16=True)
weather_train = reduce_mem_usage(weather_train,use_float16=True)
train = reduce_mem_usage(train,use_float16=True)

weather_test = reduce_mem_usage(weather_test,use_float16=True)
test = reduce_mem_usage(test,use_float16=True)

**Prepare Training and Test Dataset**

In [ ]:
# Merge training dataset
train = train.merge(building_metadata, left_on='building_id',right_on='building_id',how='left')
train = train.merge(weather_train,how='left',left_on=['site_id','timestamp'],right_on=['site_id','timestamp'])
del weather_train

# Merge test dataset
test = test.merge(building_metadata, left_on='building_id',right_on='building_id',how='left')
test = test.merge(weather_test,how='left',left_on=['site_id','timestamp'],right_on=['site_id','timestamp'])
del weather_test

In [ ]:
# Replace categorical values 

from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()

train["primary_use"] = lb_make.fit_transform(train["primary_use"])
test['primary_use']= lb_make.fit_transform(test["primary_use"])

In [ ]:
# Drop columns
train = train.drop(['timestamp'], axis = 1)
test = test.drop(['timestamp'], axis = 1)

In [ ]:
# Split train labels and training 

y_train = np.log1p(train["meter_reading"])
x_train = train.drop(['meter_reading'], axis = 1)
del train

In [ ]:
# Replace NaN values
# from sklearn.impute import SimpleImputer

# imputer  = SimpleImputer(missing_values=np.nan, strategy='mean')

# x_train = imputer.fit_transform(x_train.values)
# test = imputer.fit_transform(test.values)

In [ ]:
print(np.isnan(x_train).sum())
print(np.isnan(test).sum())

In [ ]:
# Scale Data

# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()
# scaler.fit(x_train.values)
# MinMaxScaler(copy=True, feature_range=(0, 1))

**Check Distribution of Data**

The target values is in a normal distribution, this is due to the log transformation that was performed earlier. 

**Create Baseline Model**

In [ ]:
import tensorflow as tf
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(12, activation='relu'),
    keras.layers.Dense(1, kernel_initializer='normal')
])

In [ ]:
model.compile(loss='mean_squared_error',optimizer=optimizer)
model.fit(x_train,y_train.values,epochs=10, batch_size = 100,validation_split = 1/5)

In [ ]:
from sklearn.metrics import mean_squared_log_error

predictions = model.predict(x_train)
np.sqrt(mean_squared_log_error( y_train, predictions ))